<a href="https://colab.research.google.com/github/edcote/kaggle/blob/main/Titantic_Machine_Learning_from_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"edmondcote","key":"10e43391a0bdb6ad3000b6cd932dd912"}'}

In [25]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic
!unzip -qf titanic.zip

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
/content
total 140
-rw-r--r-- 1 root root  3258 Dec 11  2019  gender_submission.csv
-rw-r--r-- 1 root root    66 Oct  7 22:23 'kaggle (1).json'
-rw-r--r-- 1 root root    66 Oct  7 22:18  kaggle.json
drwxr-xr-x 1 root root  4096 Oct  4 13:23  sample_data
-rw-r--r-- 1 root root 28629 Dec 11  2019  test.csv
-rw-r--r-- 1 root root 34877 Dec 11  2019  titanic.zip
-rw-r--r-- 1 root root 61194 Dec 11  2019  train.csv


In [28]:
import pandas as pd

In [41]:
train_data = pd.read_csv('/content/train.csv', index_col='PassengerId')
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
train_data['Embarked'] = train_data['Embarked'].fillna('S')
train_data['Sex'] = train_data['Sex'].map({'female': 0, 'male': 1}).astype(int)
train_data['Embarked_S'] = train_data['Embarked'] == 'S'
train_data['Embarked_C'] = train_data['Embarked'] == 'C'
train_data['Embarked_Q'] = train_data['Embarked'] == 'Q'

train_data['Age Group'] = pd.cut(
  train_data['Age'],
  bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90],
  labels=["0_10", "10_20", "20_30", "30_40", "40_50", "50_60", "60_70", "70_80", "80_90"]
)

train_data = train_data.drop(['Ticket', 'Name', 'Cabin', 'Embarked'], axis=1)

train_data.Age.describe()

,Age
count,891.000000
mean,29.361582
std,13.019697
min,0.420000
25%,22.000000
50%,28.000000
75%,35.000000
max,80.000000


In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

#train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
#train_data['Sex'] = train_data['Sex'].map({'female': 0, 'male': 1}).astype(int)
#train_data = train_data.drop(['Ticket', 'Name', 'Cabin'], axis=1)


In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

# Data cleaning

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

#train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
#train_data['Sex'] = train_data['Sex'].map({'female': 0, 'male': 1}).astype(int)
#train_data = train_data.drop(['Ticket', 'Name', 'Cabin'], axis=1)


In [ ]:
train_data.describe()